<a href="https://colab.research.google.com/github/TekyaygilFethi/GoogleInterviewQuestions-CalendarProblem/blob/main/Google_Interview_Question_Calendar_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This question is taken from https://www.youtube.com/watch?v=3Q_oYDQ2whs

"""
Assume two person have their own calendar. Our goal is to determine that 
the times they could have a meeting with a given duration and given daily bounds for 
each person.
"""

In [ ]:
# Assume this is our input
input={
    "Person1Calendar":[["09:00","10:00"],["12:00","13:00"],["13:00","13:15"],["16:00","18:00"]],
    "Person1DailyBounds":["09:00","20:00"],
    "Person2Calendar":[["10:10","11:30"],["12:30","14:30"],["14:30","15:00"],["15:00","15:20"],["15:40","16:50"],["16:00","17:00"]],
    "Person2DailyBounds":["10:00","18:30"],
    "MeetingDuration":15
}

In [ ]:
# We merge our calendars into single list
big_list = input["Person1Calendar"]
big_list.extend(input["Person2Calendar"])

In [ ]:
# We get daily bounds for each person, zip them and find max of the start times and
# min of the end times in order to get the time bounds possible for meeting times

daily_bound_list=input["Person1DailyBounds"]
daily_bound_list = list(zip(daily_bound_list,input["Person2DailyBounds"]))

start_time = max(list(daily_bound_list)[0])
end_time = min(list(daily_bound_list)[1])

In [ ]:
# We get our given duration
duration = input["MeetingDuration"]

In [ ]:
# We are sorting our merged list according to their start times
import itertools
big_list.sort()
big_list = list(big_list for big_list,_ in itertools.groupby(big_list))

In [ ]:
# We parse the start and end times to get minutes for comparison and difference betwwen them
# in terms of minutes
def GetMinutesFromTimes(time1,time2):
  h1, m1=time1.split(":")
  h2, m2=time2.split(":")

  t1=int(h1)*60+int(m1)
  t2=int(h2)*60+int(m2)

  return (t1,t2)

In [ ]:
# Compare times to determine which is earlier or not
def CompareTimes(time1,time2):
  t1, t2 = GetMinutesFromTimes(time1,time2)

  if t1>t2:
    return 1
  elif t1<t2:
    return -1
  else:
    return 0

In [ ]:
# Check for the time differences for two given time if they have the given duration or 
# more minutes between them
def IsTimeDifferenceAppropriate(time1,time2,duration):
  t1, t2 = GetMinutesFromTimes(time1,time2)

  return t2-t1>=duration

In [ ]:
# Set the start time bound if necessary, 
# if there is a time interval earlier than start time bound delete it
def CompareStartTimeOfMeetingsForAvailability(temp_list):
  index = 0
  while index < len(temp_list):
    if CompareTimes(start_time,temp_list[index][0])>0:
      if CompareTimes(start_time,temp_list[index][1])<0:
        temp_list[index][0]=start_time
      else:
        temp_list.pop(index)
        index-=1
      index+=1
    else:
      break

# Set the end time bound if necessary, 
# if there is a time interval later than end time bound delete it
def CompareEndTimeOfMeetingsForAvailability(temp_list):
  index=len(temp_list)-1

  while index >= 0:
    if CompareTimes(temp_list[index][1],end_time)>0:
      if CompareTimes(temp_list[index][0],end_time)>=0:
        temp_list.pop(index)
        
      else:
        temp_list[index][1]=end_time

      index-=1
    else:
      break

# Merge the meeting times if their time interval is between each other and make new internal
# to contain the earliest time for start and latest time for end time
def CompareTimesForCommonMeetingTimes(temp_list):
  index = 0
  while index < len(temp_list)-1:
    if CompareTimes(temp_list[index][1], temp_list[index+1][0])>=0:
      if CompareTimes(temp_list[index][1], temp_list[index+1][1])<0:
        temp_list[index][1]=temp_list[index+1][1]

      temp_list.pop(index+1)
      index-=1
    
    index+=1

# Our main function. It computes the availble time schedule
def ScheduleAvailability(liste):
  available_time_blocks=[]
  temp_list=liste

  CompareStartTimeOfMeetingsForAvailability(temp_list)

  CompareEndTimeOfMeetingsForAvailability(temp_list)

  CompareTimesForCommonMeetingTimes(temp_list)
  
  # If there is a space between the earliest time interval of the merged calendar list 
  # and earliest time bound of the day then check their time difference and if it's equal 
  # or greater than the given meeting duration add the new time interval into available_time_blocks list
  if CompareTimes(temp_list[0][0],start_time)==1 and IsTimeDifferenceAppropriate(temp_list[0][0],start_time,duration):
    available_time_blocks.append([start_time,temp_list[0][0]])

  # Fill the blanks between the merged calendar list with checking if they have appropriate
  # duration. If so, add the new time interval into the available_time_blocks list
  index = 0
  while index < len(temp_list)-1:
    if IsTimeDifferenceAppropriate(temp_list[index][1],temp_list[index+1][0],duration):
      available_time_blocks.append([temp_list[index][1],temp_list[index+1][0]])
    
    index+=1


  # If there is a space between the latest time interval of the merged calendar list 
  # and earliest time bound of the day then check their time difference and if it's equal 
  # or greater than the given meeting duration add the new time interval into available_time_blocks list
  if CompareTimes(temp_list[-1][1],end_time)==-1 and IsTimeDifferenceAppropriate(temp_list[-1][1],end_time,duration):
    available_time_blocks.append([temp_list[-1][1],end_time])
  
  return available_time_blocks
    

available_time_blocks = ScheduleAvailability(big_list[:])

print(available_time_blocks)